# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List

In [2]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
class DecisionTreeLeaf:
    def __init__(self, y):
        values, counts = np.unique(y, return_counts=True)
        counts = counts / y.shape[0]
        self.probabilities = dict(zip(values, counts))

class DecisionTreeNode:
    def __init__(self, split_dim: int, 
                 left: Union['DecisionTreeNode', DecisionTreeLeaf], 
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.left = left
        self.right = right
        
class DecisionTree:
    def __init__(self, X, y, criterion : str = "gini", 
                 max_depth : Optional[int] = None, 
                 min_samples_leaf: int = 1,
                 max_features="auto"):
        n, m = X.shape
        self.critetion = gini if criterion == "gini" else entropy
        self.max_depth = max_depth if max_depth is not None else np.inf
        self.min_samples_leaf = min_samples_leaf
        self.max_features = int(np.ceil(np.sqrt(m))) if max_features == 'auto' else max_features
        inds = np.random.choice(n, n, replace=True)
        self.oob = np.setdiff1d(np.arange(n), inds)
        self.root = self.build_tree(X[inds], y[inds], 0)
        
        
    def build_tree(self, X: np.ndarray, y: np.ndarray, depth):
        if depth >= self.max_depth:
            return DecisionTreeLeaf(y)
        
        split_dim, split_gain = -1, -np.inf
        for i, col in enumerate(X.T):
            values = np.unique(col)
            for value in values:
                mask = col == 1
                left = y[mask]
                right = y[~mask]
                if left.shape[0] >= self.min_samples_leaf and right.shape[0] >= self.min_samples_leaf:
                    g = gain(left, right, self.critetion)
                    if g > split_gain:
                        split_dim, split_gain = i, g
        
        if split_dim == -1:
            return DecisionTreeLeaf(y)
        
        mask = X[:, split_dim] == 1
        left = self.build_tree(X[mask], y[mask], depth + 1)
        right = self.build_tree(X[~mask], y[~mask], depth + 1)
        return DecisionTreeNode(split_dim, left, right)
        
    def get_probabilities(self, x, node):
        if isinstance(node, DecisionTreeLeaf):
            return node.probabilities
        if x[node.split_dim] == 1:
            return self.get_probabilities(x, node.left)
        else:
            return self.get_probabilities(x, node.right)
    
    def predict_proba(self, X: np.ndarray) ->  List[Dict[Any, float]]:
        return [self.get_probabilities(x, self.root) for x in X]
    
    def predict(self, X : np.ndarray) -> list:
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [4]:
from collections import Counter

In [5]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
    
    def fit(self, X, y):
        self.X, self.y = X, y
        self.trees = [DecisionTree(X, y,
                                   self.criterion,
                                   self.max_depth,
                                   self.min_samples_leaf,
                                   self.max_features) for _ in range(self.n_estimators)]
    def predict(self, X):
        pred = np.array([tree.predict(X) for tree in self.trees]).T
        return np.apply_along_axis(lambda r: Counter(r).most_common(1)[0][0], 1, pred)

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [6]:
def feature_importance(rfc):
    X, y = rfc.X, rfc.y
    n, m = X.shape
    importance = np.zeros((rfc.n_estimators, m))
    for i, t in enumerate(rfc.trees):
        X_oob, y_oob = X[t.oob], y[t.oob]
        err_oob = np.mean(t.predict(X_oob) != y_oob)
        for j in range(m):
            X_shuffled = X_oob.copy()
            np.random.shuffle(X_shuffled[:, j])
            err_oob_j = np.mean(t.predict(X_shuffled) != y_oob)
            importance[i, j] = err_oob_j - err_oob
    return np.mean(importance, axis=0)


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [7]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.         0.         0.19486684 0.19329859 0.44416171 0.        ]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [8]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [9]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [10]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7049180327868853
Most important features:
1. ovsyanochan
2. 4ch
3. styd.pozor
4. mudakoff
5. rhymes
6. xfilm
7. tumblr_vacuum
8. iwantyou
9. rapnewrap
10. pravdashowtop
11. reflexia_our_feelings
12. bot_maxim
13. pixel_stickers
14. ne.poverish
15. dayvinchik
16. bestad
17. bog_memes
18. dzenpub
19. ne1party
20. fuck_humor


#### Пол

In [11]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8461538461538461
Most important features:
1. 40kg
2. mudakoff
3. zerofat
4. modnailru
5. girlmeme
6. thesmolny
7. 9o_6o_9o
8. be.beauty
9. i_d_t
10. reflexia_our_feelings
11. woman.blog
12. 4ch
13. femalemem
14. sh.cook
15. cook_good
16. rapnewrap
17. be.women
18. combovine
19. bot_maxim
20. psy.people


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [12]:
X, y = synthetic_dataset(1000)
cbc = CatBoostClassifier(loss_function="MultiClass", verbose=False)
cbc.fit(X, y)
y_pred = cbc.predict(X).flatten()
print("Accuracy:", np.mean(y_pred == y))
print("Importance:", cbc.feature_importances_)

Accuracy: 1.0
Importance: [1.24975988e-02 7.01002161e-03 2.78613400e+01 2.78602123e+01
 4.42540747e+01 4.86545539e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [13]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [14]:
cbc = CatBoostClassifier(loss_function='MultiClass', verbose=False)
cbc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(cbc.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(cbc.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7629255989911727
Most important features:
1. mudakoff
2. ovsyanochan
3. 4ch
4. styd.pozor
5. rhymes
6. dayvinchik
7. leprum
8. rapnewrap
9. xfilm
10. fuck_humor


#### Пол

In [15]:
cbc = CatBoostClassifier(loss_function="MultiClass", verbose=False)
cbc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(cbc.predict(X_test).flatten() == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(cbc.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8764186633039092
Most important features:
1. 40kg
2. mudakoff
3. modnailru
4. girlmeme
5. femalemem
6. 9o_6o_9o
7. academyofman
8. rapnewrap
9. igm
10. i_d_t
